In [63]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
import langchain_core, langchain_ollama
import pprint

#langchain_core : 0.3.10
#langchain_ollama : 0.2.0
print("langchain_core : " + langchain_core.__version__)
print("langchain_ollama : " + langchain_ollama.__version__)

langchain_core : 0.3.10
langchain_ollama : 0.2.0


In [49]:
# model="llama3.2:3b" 에서 했으나, tool calling 이 제대로 적용되지 않음.
# 버전은 낮지만, 더 큰 모델을 사용했더니 성공. llama3.1:8b
llm = ChatOllama(
    model="llama3.1:8b",
    temperature=0,
    base_url="http://host.docker.internal:11434"
)

In [51]:
ai_msg = llm.invoke(input="What is 3 * 12? Also, what is 11 + 49?")
print(ai_msg.content)

Two simple math questions!

**1. 3 * 12 = 36**

Multiplying 3 by 12 gives us a product of 36.

**2. 11 + 49 = 60**

Adding 11 and 49 together gives us a sum of 60.


In [52]:
from langchain_core.tools import tool

@tool
def add(a:int, b:int) -> int:
    """Adds a and b."""
    return a + b + 10

@tool
def multiply(a:int, b:int) -> int:
    """Multiplies a and b."""
    return a * b * 10   

In [53]:
tools = [add, multiply]
llm_with_tools = llm.bind_tools(tools)

In [54]:
from langchain_core.messages import HumanMessage

query = "What is 3 * 12? Also, what is 11 + 49?"

In [72]:
messages = [HumanMessage(query)]

In [73]:
ai_msg = llm_with_tools.invoke(messages)
pprint.pprint(ai_msg.tool_calls)

[{'args': {'a': 3, 'b': 12},
  'id': '66e773b5-9012-4343-b74e-e674bf9322fa',
  'name': 'multiply',
  'type': 'tool_call'},
 {'args': {'a': 11, 'b': 49},
  'id': '502862d1-0d5b-48be-892e-8355ac2f292f',
  'name': 'add',
  'type': 'tool_call'}]


In [74]:
messages.append(ai_msg)

In [75]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply":multiply}[tool_call["name"]]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

In [84]:
pprint.pp(messages)

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2024-10-14T10:05:58.477577Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'multiply', 'arguments': {'a': 3, 'b': 12}}}, {'function': {'name': 'add', 'arguments': {'a': 11, 'b': 49}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 5185118709, 'load_duration': 32918959, 'prompt_eval_count': 233, 'prompt_eval_duration': 1599681000, 'eval_count': 43, 'eval_duration': 3551051000}, id='run-ad7f5247-74c9-4d55-97e5-1e791446a9b9-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': '66e773b5-9012-4343-b74e-e674bf9322fa', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': '502862d1-0d5b-48be-892e-8355ac2f292f', 'type': 'tool_call'}], usage_metadata={'input_tokens': 233, 'output_tokens': 43, 'total_to

In [85]:
ai_final_msg = llm_with_tools.invoke(messages)

In [86]:
print(ai_final_msg.content)

The result of the multiplication is 360, and the result of the addition is 70.
